In [108]:
import tensorflow as tf
import numpy as np
import chess
import chesslib




In [109]:
#input_data = np.random.random((9, 8, 12))
#input_data = np.empty((9,8,12))
output_data = np.array((-0.122, 0.122, -0.194, -0.06, 0.02))

board = chess.Board()

board.set_fen('r2qkb2/pbN2ppr/2p2n2/4P2P/PQp1N2P/5P2/1PR1P3/5K2 b q -')
new_train_array1 = chesslib.board_to_array(board)


board.set_fen('4qbr1/3k1p1Q/3p4/P1pPpPp1/p1R3NP/5P2/1B1R2P1/5KN1 w - -')
new_train_array2 = chesslib.board_to_array(board)


board.set_fen('rn1k1b1r/5q1Q/3p3p/1pp1p3/P1P1b1P1/2NPpPP1/1P4B1/R1BRK3 w - -')
new_train_array3 = chesslib.board_to_array(board)


board.set_fen('1n1qr3/1Qpkp3/b7/pp3pPp/3P1P1P/PpBB4/8/RNbK3R b - -')
new_train_array4 = chesslib.board_to_array(board)


board.set_fen('rnbqkb1r/p1p1pp2/1p4pn/3p3p/7P/PP1P1P2/2P1P1P1/RNBQKBNR b KQkq -')
new_train_array5 = chesslib.board_to_array(board)

input_data = np.stack((new_train_array1, new_train_array2, new_train_array3, new_train_array4, new_train_array5), axis = 0)


print(input_data.shape)

(5, 9, 8, 12)


In [110]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [111]:

# Define the model
model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=(8, 12, 9)),  # Flatten the 12x9 matrix to a 1D array
    tf.keras.layers.Dense(64, activation='relu'),  # First dense layer
    tf.keras.layers.Dense(28, activation='sigmoid'),    # Output layer with tanh activation
    tf.keras.layers.Dense(1, activation='tanh')    # Output layer with tanh activation
])

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy')

# Summary of the model
model.summary()

Model: "sequential_14"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ flatten_14 (Flatten)            │ (None, 864)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_38 (Dense)                │ (None, 64)             │        55,360 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_39 (Dense)                │ (None, 28)             │         1,820 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_40 (Dense)                │ (None, 1)              │            29 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 57,209 (223.47 KB)

 Trainable params: 57,209 (223.47 KB)

 Non-trainable params: 0 (0.00 B)

In [112]:
model.fit(input_data, output_data, epochs=50)

Epoch 1/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 568ms/step - loss: 2.0759
Epoch 2/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step - loss: 1.9396
Epoch 3/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - loss: 1.8103
Epoch 4/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - loss: 1.6853
Epoch 5/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - loss: 1.5601
Epoch 6/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - loss: 1.4356
Epoch 7/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - loss: 1.3127
Epoch 8/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step - loss: 1.1919
Epoch 9/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step - loss: 1.0727
Epoch 10/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - loss: 0.9539
Epoch 11/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step - loss: 0.8351
Epoch 12/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - loss: 0.7179
Epoch 13/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - loss: 0.6020
Epoch 14/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - loss: 0.4878
Epoch 15/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step - loss: 0.3787
Epoch 16/50
1/1 ━━━━━━━━━━━━━━━━━

In [113]:
board.set_fen('6k1/4r2r/3p3b/N4p1p/Pq2NQ2/2PP1KB1/8/2R1RB2 w - -')
new_test_array1 = chesslib.board_to_array(board)


board.set_fen('1n1kr3/3p2np/1rb4b/pp3pp1/Pp1Pp2P/2BRPN2/2P1KPB1/R3N2Q b - -')
new_test_array2 = chesslib.board_to_array(board)

input_data = np.stack((new_test_array1, new_test_array2), axis = 0)

In [114]:

model.predict(input_data)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step


array([[ 0.17802954],
       [-0.11991939]], dtype=float32)